In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pydantic import BaseModel

class SimpleModelParams(BaseModel):
    birth_rate: float
    ntaxa: int
    J: int
    colrate: float
    Ne_scaling: float

class SimpleModelOutput(BaseModel):
    local_S: float
    abund_h1: float
    abund_h2: float
    abund_h3: float
    pi_h1: float
    pi_h2: float
    pi_h3: float
    trait_h1: float
    trait_h2: float
    trait_h3: float

In [3]:
import iBioGen
import pandas as pd
from typing import List
import fastapi
from fastapi import HTTPException
import asyncio

app = fastapi.FastAPI()

params = SimpleModelParams(birth_rate=0.1619652658700943, ntaxa=84, J=11700, colrate=0.9727293252944946, Ne_scaling=744.97509765625)

def _simulate(params: SimpleModelParams) -> List[SimpleModelOutput]:
    core = iBioGen.Core("python-testing", quiet=True)
    
    core.set_param("meta_stop_criterion", "time")
    core.set_param("time", 100)
    core.set_param("birth_rate", params.birth_rate)
    core.set_param("ntaxa", params.ntaxa)
    core.set_param("J", params.J)
    core.set_param("colrate", params.colrate)
    # core.set_param("local_stop_time", params.local_stop_time)
    core.set_param("Ne_scaling", params.Ne_scaling)
    
    core.simulate(nsims=5, force=True)
    results = core.load_sims()
    summary: pd.DataFrame = results[0]
    output = []
    for _, row in summary.iterrows():
        output.append(SimpleModelOutput(
            local_S=row['local_S'],
            abund_h1=row['abund_h1'],
            abund_h2=row['abund_h2'],
            abund_h3=row['abund_h3'],
            pi_h1=row['pi_h1'],
            pi_h2=row['pi_h2'],
            pi_h3=row['pi_h3'],
            trait_h1=row['trait_h1'],
            trait_h2=row['trait_h2'],
            trait_h3=row['trait_h3']
        ))
    return output

@app.post("/simulate", response_model=List[SimpleModelOutput])
async def simulate(params: SimpleModelParams) -> List[SimpleModelOutput]:
    return _simulate(params)
    # try:
    #     result = await asyncio.wait_for(_simulate(params), timeout=4)
    #     return result
    # except asyncio.TimeoutError:
    #     raise HTTPException(status_code=408, detail="Request timed out")
    # except Exception as e:
    #     raise HTTPException(status_code=500, detail=str(e))
    # loop = asyncio.get_event_loop()
    # try:
    #     result = await asyncio.wait_for(
    #         loop.run_in_executor(None, _simulate, params),  # None uses the default ThreadPoolExecutor
    #         timeout=4
    #     )
    #     return result
    # except asyncio.TimeoutError:
    #     raise HTTPException(status_code=408, detail="Request timed out")
    # except Exception as e:
    #     raise HTTPException(status_code=500, detail=str(e))

In [ ]:
import uvicorn
import threading

# Start the FastAPI app in a separate thread
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_app, daemon=True)
thread.start()

INFO:     Started server process [2293]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


    Generating 5 simulation(s).
  [####################] 100%  Finished 5 simulations in   0:00:30 | 
 INFO:     127.0.0.1:65155 - "POST /simulate HTTP/1.1" 200 OK
    Generating 5 simulation(s).
  [####################] 100%  Finished 5 simulations in   0:00:20 | 
 INFO:     127.0.0.1:65171 - "POST /simulate HTTP/1.1" 200 OK


,0,1,2,3,4,5,6,7,8,9
0,"(local_S, 75.0)","(abund_h1, 10.669460912894277)","(abund_h2, 5.631107307950611)","(abund_h3, 4.63356413266069)","(pi_h1, 12.835636479809214)","(pi_h2, 8.255900825652516)","(pi_h3, 6.652512575494367)","(trait_h1, 10.023769760921187)","(trait_h2, 5.531587382062691)","(trait_h3, 4.652379715047779)"
1,"(local_S, 81.0)","(abund_h1, 15.7673817179996)","(abund_h2, 7.630924795170793)","(abund_h3, 5.925326009172812)","(pi_h1, 16.271909158750926)","(pi_h2, 11.018141640232388)","(pi_h3, 9.232489967805147)","(trait_h1, 24.798473125300042)","(trait_h2, 12.000899752119336)","(trait_h3, 8.626890789903118)"
2,"(local_S, 76.0)","(abund_h1, 15.83971665394956)","(abund_h2, 9.459612795085121)","(abund_h3, 7.5667180702245584)","(pi_h1, 10.669048003715021)","(pi_h2, 6.753085344383474)","(pi_h3, 5.631791406610793)","(trait_h1, 17.733702067600337)","(trait_h2, 11.058284431590705)","(trait_h3, 8.996621427742037)"
3,"(local_S, 73.0)","(abund_h1, 16.24378931744502)","(abund_h2, 8.674245304539577)","(abund_h3, 6.746548740062722)","(pi_h1, 17.98985058443275)","(pi_h2, 11.316394510529516)","(pi_h3, 8.915946750511248)","(trait_h1, 14.777663381933491)","(trait_h2, 8.395445397161058)","(trait_h3, 6.8779781904834225)"
4,"(local_S, 81.0)","(abund_h1, 25.658595074310956)","(abund_h2, 17.39399813671131)","(abund_h3, 14.271169051230192)","(pi_h1, 22.129429223560898)","(pi_h2, 16.65759062246771)","(pi_h3, 14.310692448350038)","(trait_h1, 32.00126654736128)","(trait_h2, 20.86716917125413)","(trait_h3, 16.773319450619578)"


In [75]:
results[0]

,birth_rate,meta_stop_criterion,ntaxa,time,process,ClaDS,abundance_mean,abundance_sigma,growth_rate_mean,growth_rate_sigma,...,local_S,abund_h1,abund_h2,abund_h3,pi_h1,pi_h2,pi_h3,trait_h1,trait_h2,trait_h3
0,0.161965,taxa,84,4,abundance,True,50000,0.1,0,0.01,...,82,27.348666,15.441795,11.843334,25.091403,15.640953,12.261934,31.407041,18.296986,14.056508
1,0.161965,taxa,84,4,abundance,True,50000,0.1,0,0.01,...,44,4.039505,2.386702,2.035565,6.421666,4.384805,3.837671,12.826735,5.161098,4.295411
2,0.161965,taxa,84,4,abundance,True,50000,0.1,0,0.01,...,74,20.075268,13.376451,10.754319,19.742472,14.941765,12.581970,31.661779,19.664887,15.595536
3,0.161965,taxa,84,4,abundance,True,50000,0.1,0,0.01,...,76,33.240502,24.560190,20.898421,27.833620,19.945768,16.396361,32.674623,24.324366,20.818048
4,0.161965,taxa,84,4,abundance,True,50000,0.1,0,0.01,...,78,18.770440,12.747745,10.399283,14.074713,10.666958,9.502700,20.243085,14.072616,12.055664
